# Setup

Mount Google Drive:



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
gdrive = True

Copy dataset if needed:

In [ ]:
!cp -r '/content/drive/MyDrive/BIAI/data' '/content/data'
local_dataset = True

# Preparations

Prepare the Colab environment for imports:

In [1]:
FOLDER_HOME = '/content/drive/MyDrive/BIAI' if 'gdrive' in locals() and gdrive else '.'

import sys
sys.path.append(FOLDER_HOME)

Import the modules and define constants:

In [6]:
from data import Loader, TestLogger, TrainLogger
from network import Network, Trainer, Tester, DEVICE

DATA_HOME = FOLDER_HOME + '/data'
if 'local_dataset' in locals() and local_dataset:
  DATA_HOME = '/content/data'

IMG_DIM = 320
LEARNING_RATE = 0.001
EPOCHS = 150

MODEL_OUTPUT_PATH = FOLDER_HOME + '/model.pt'
MODEL_INPUT_PATH = None

Define functions for training and testing:

In [7]:
def train(net, trainer, train_data, train_logger, epoch):
  print(f'* Training epoch ({DEVICE}): {epoch}/{EPOCHS}...')
  
  net.train()
  train_logger.start(epoch)
  loss = trainer.train(net, train_data)
  train_logger.log_epoch(loss)
  
  print(f'* Finished training epoch: {epoch}/{EPOCHS}')

def test(net, tester, test_data, test_logger):
  print(f'* Testing...')
  
  net.eval()
  test_logger.start()
  correct, total = tester.test(net, test_data)
  test_logger.log_test_result(correct, total)
  
  print(f'Correct: {correct}/{total}, ({100 * correct / total:.2f}%)')

Create all the components:

In [8]:
loader = Loader(DATA_HOME)
train_folder, train_data = loader.load_train()
test_folder, test_data = loader.load_test()
classes = train_folder.classes

net = None
if MODEL_INPUT_PATH:
  net = Network.load(MODEL_INPUT_PATH)
else:
  net = Network(IMG_DIM, len(classes), classes)

trainer = Trainer(LEARNING_RATE)
trainer.verbose = True
trainer.report_freq = 4
tester = Tester()

train_logger = TrainLogger(FOLDER_HOME + '/train.csv')
test_logger = TestLogger(FOLDER_HOME + '/test.csv')

# Main loop

In [9]:
train(net, trainer, train_data, train_logger, 1)
net.save(MODEL_OUTPUT_PATH)
test(net, tester, test_data, test_logger)

train_logger.append = True
test_logger.append = True
for epoch in range(1, EPOCHS):
  train(net, trainer, train_data, train_logger, epoch + 1)
  net.save(MODEL_OUTPUT_PATH)
  test(net, tester, test_data, test_logger)

net.save(MODEL_OUTPUT_PATH)

* Training epoch (cuda): 1/150...
Progress: 1.11% (avg. loss: 3.512)
Progress: 25.56% (avg. loss: 3.377)
Progress: 50.00% (avg. loss: 3.334)
Progress: 74.44% (avg. loss: 3.035)
Progress: 98.89% (avg. loss: 2.875)
* Finished training epoch: 1/150
* Testing...
Correct: 56/329, (17.02%)
* Training epoch (cuda): 2/150...
Progress: 1.11% (avg. loss: 2.802)
Progress: 25.56% (avg. loss: 3.008)
Progress: 50.00% (avg. loss: 2.946)
Progress: 74.44% (avg. loss: 2.677)
Progress: 98.89% (avg. loss: 2.603)
* Finished training epoch: 2/150
* Testing...
Correct: 55/329, (16.72%)
* Training epoch (cuda): 3/150...
Progress: 1.11% (avg. loss: 3.022)
Progress: 25.56% (avg. loss: 2.683)
Progress: 50.00% (avg. loss: 2.466)
Progress: 74.44% (avg. loss: 2.531)
Progress: 98.89% (avg. loss: 2.440)
* Finished training epoch: 3/150
* Testing...
Correct: 68/329, (20.67%)
* Training epoch (cuda): 4/150...
Progress: 1.11% (avg. loss: 2.812)
Progress: 25.56% (avg. loss: 2.076)
Progress: 50.00% (avg. loss: 2.396)
Prog